# Data Project - Data Engineer - ETL




### In this Project I will:
*   Extract using API and Scraping   
*   Implement the ETL process
*   Extract bank and market cap data from the JSON file `bank_market_cap.json`
*   Transform the market cap currency using the exchange rate data
*   Load the transformed data into a seperate CSV
*   Logging each step in the Pipeline

## Imports

Import any additional libraries you may need here.


In [117]:
import glob
import pandas as pd
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import html5lib

As the exchange rate fluctuates, we will download the same dataset to make marking simpler. This will be in the same format as the dataset you used in the last section


## Extract Data Using an API


In [119]:
# API Url and APIKey
url = "https://api.apilayer.com/exchangerates_data/latest?base=EUR&apikey=fWZUH7CEHQBZPwsD7AZUAW7a7SyMLSun"
data = requests.get(url)
rates_dic = data.json()

# Turn the data into a dataframe
rates = pd.DataFrame().from_dict(rates_dic)

# Drop unnescessary columns
rates.drop(rates.columns[[0,1,2,3]], axis = 1, inplace =True)
print(rates.columns)

# Save the Dataframe
rates.to_csv('exchange_rates.csv')

Index(['rates'], dtype='object')


## Extract Data Using Web Scraping

### Scraping the Data

Using the contents and `beautiful soup` load the data from the `By market capitalization` table into a `pandas` dataframe. The dataframe should have the country `Name` and `Market Cap (US$ Billion)` as column names.  Display the first five rows using head.

In [120]:
# Gather the contents of the webpage
data = requests.get('https://en.wikipedia.org/wiki/List_of_largest_banks')
html_data = data.content
sou = 'By market capitalization'
soup = BeautifulSoup(html_data, "html.parser")
result = soup.find_all("tbody")
print(result)
# Extract data from page content
market_cap = pd.DataFrame(columns = ['Name','Market Cap(US$ Billion'])
i = 0
name= []
market = []
for row in result[3].find_all('tr'):
    rank = row.find_all('td')
    if i == 0:
        i += 1
        pass
    else:
        #print(rank[1].text.strip())
        name.append(rank[1].text.strip())
        market.append(rank[2].text.strip())
market_cap['Name'] = name
market_cap['Market Cap(US$ Billion'] = market
market_cap.head()
# Save the data as json file 
market_cap.to_json('bank_market_cap.json')

[<tbody><tr><td class="mbox-image"><div style="width:52px"><a class="image" href="/wiki/File:Wiki_letter_w.svg"><img alt="Wiki letter w.svg" data-file-height="44" data-file-width="44" decoding="async" height="44" src="//upload.wikimedia.org/wikipedia/en/thumb/6/6c/Wiki_letter_w.svg/44px-Wiki_letter_w.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/6/6c/Wiki_letter_w.svg/66px-Wiki_letter_w.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/6/6c/Wiki_letter_w.svg/88px-Wiki_letter_w.svg.png 2x" width="44"/></a></div></td><td class="mbox-text"><div class="mbox-text-span">This article <b>is missing information</b> about Revenue and Employment.<span class="hide-when-compact"> Please expand the article to include this information. Further details may exist on the <a href="/wiki/Talk:List_of_largest_banks" title="Talk:List of largest banks">talk page</a>.</span> <span class="date-container"><i>(<span class="date">September 2020</span>)</i></span></div></td></tr></tbody>, <tbody

In [121]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2022-05-20 02:34:52--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: ‘bank_market_cap_1.json.2’

bank_market_cap_1.j 100%[===================>]   2.75K  --.-KB/s    in 0s      

2022-05-20 02:34:53 (538 MB/s) - ‘bank_market_cap_1.json.2’ saved [2815/2815]

--2022-05-20 02:34:53--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20

## Extract


### JSON Extract Function

This function will extract JSON files.


In [122]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process, lines= False)
    return dataframe

## Extract Function

Define the extract function that finds JSON file `bank_market_cap_*.json` and calls the function created above to extract data from them. Store the data in a `pandas` dataframe. Use the following list for the columns.


In [123]:
columns=['Name','Market Cap (US$ Billion)']

In [124]:
def extract():
    json_data = pd.DataFrame(columns = columns)
    for file in glob.glob('*.json'):
       # print(file, '  ',extract_from_json('bank_market_cap_1.json'))
        json_data = json_data.append(extract_from_json('bank_market_cap_1.json'), ignore_index= True)
    return json_data


Load the file <code>exchange_rates.csv</code> as a dataframe and find the exchange rate for British pounds with the symbol <code>GBP</code>


In [125]:
def extract_from_csv(csv_file):
    dataframe = pd.read_csv(csv_file, index_col = 0)
    return dataframe

exchange_rates = extract_from_csv('exchange_rates.csv')
exchange_rate = exchange_rates.loc['GBP']
print(exchange_rate)

rates    0.848714
Name: GBP, dtype: float64


## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file find the exchange rate of USD to GBP. Write a transform function that

1.  Changes the `Market Cap (US$ Billion)` column from USD to GBP
2.  Rounds the Market Cap (US$ Billion)\` column to 3 decimal places
3.  Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


In [126]:
def transform(bank_market_cap, exchange_rate):
    bank_market_cap['Market Cap (US$ Billion)'] =[ round(x * exchange_rate,3) for x in bank_market_cap.loc[:,'Market Cap (US$ Billion)']]
    #bank_market_cap['Market Cap (US$ Billion)'] = [round(x,3) for x in bank_market_cap.loc[:,'Market Cap (US$ Billion)']]
    bank_market_cap.rename( columns = {'Market Cap (US$ Billion)' : 'Market Cap (GBP$ Billion)'}, inplace = True)
    return bank_market_cap



## Load

Create a function that takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`.


In [127]:
def load(data, target):
    data.to_csv(target, index = False)

## Logging Function


Write the logging function <code>log</code> to log your data:


In [128]:
def log(message):
    # Write your code here
    timestamp_format = '%Y - %h - %d %H:%M:%S'
    cur_date = datetime.now()
    timestamp = cur_date.strftime(timestamp_format)
    with open('log.csv','a') as log_f:
      log_f.write(message + ' '+ timestamp+' ')


## Running the ETL Process


In [129]:
# Write your code here
log('ETL Job Started')
log('Extract Phase Started')
# Call the function here
extracted_data = extract()
# Print the rows here
print(extracted_data.head())
# Write your code here
log('Extract Phase ended')
# Write your code here
log('Transform Phase Started')
# Call the function here
transformed_data = transform(extracted_data,exchange_rate)
# Print the first 5 rows here
print(transformed_data.head())
# Write your code here
log('Transformed phase Ended')
# Write your code here
log('Load phase Started')
# Write your code here
load(transformed_data,'bank_market_cap_gbp.csv')
# Write your code here
log('Load phase Ended')

                                      Name  Market Cap (US$ Billion)
0                           JPMorgan Chase                   390.934
1  Industrial and Commercial Bank of China                   345.214
2                          Bank of America                   325.331
3                              Wells Fargo                   308.013
4                  China Construction Bank                   257.399
                                      Name  \
0                           JPMorgan Chase   
1  Industrial and Commercial Bank of China   
2                          Bank of America   
3                              Wells Fargo   
4                  China Construction Bank   

                    Market Cap (GBP$ Billion)  
0  rates    331.791
Name: GBP, dtype: float64  
1  rates    292.988
Name: GBP, dtype: float64  
2  rates    276.113
Name: GBP, dtype: float64  
3  rates    261.415
Name: GBP, dtype: float64  
4  rates    218.458
Name: GBP, dtype: float64  


## Authors


Abdelrahman Mamdouh